# 🤖 Building AI Agents with AI Beast

Learn to create autonomous AI agents that can use tools and take actions.

## What You'll Learn

1. Agent architecture concepts
2. Define custom tools
3. Build a ReAct agent
4. Handle agent conversations
5. Multi-tool workflows

## Prerequisites

- Completed previous tutorials
- Understanding of function calling

## 1. Understanding Agents

Agents are LLMs that can:
- **Reason** about tasks
- **Plan** multi-step solutions
- **Act** using tools
- **Observe** results and adapt

```
┌───────────────────────────────────────────────┐
│                    Agent                       │
│  ┌─────────┐   ┌─────────┐   ┌─────────┐     │
│  │  Think  │──▶│  Plan   │──▶│   Act   │     │
│  └─────────┘   └─────────┘   └────┬────┘     │
│       ▲                           │          │
│       │       ┌─────────┐         │          │
│       └───────│ Observe │◀────────┘          │
│               └─────────┘                    │
└───────────────────────────────────────────────┘
                     │
              ┌──────┴──────┐
              ▼             ▼
        ┌─────────┐   ┌─────────┐
        │  Tool 1 │   │  Tool 2 │
        └─────────┘   └─────────┘
```

In [ ]:
import sys
sys.path.insert(0, '..')

from modules.agent import Agent, Tool, ToolResult
from modules.llm import LLMClient
from typing import Any
import json

# Initialize LLM
llm = LLMClient()
print("Ready to build agents!")

## 2. Defining Tools

Tools are functions that agents can call to interact with the world.

In [ ]:
from dataclasses import dataclass
from typing import Callable, Dict, Any, Optional

@dataclass
class ToolDefinition:
    """Defines a tool for the agent."""
    name: str
    description: str
    parameters: Dict[str, Any]
    function: Callable
    
    def to_schema(self) -> Dict[str, Any]:
        """Convert to OpenAI-style function schema."""
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": self.parameters,
            }
        }

# Define some tools
def calculator(expression: str) -> str:
    """Evaluate a math expression."""
    try:
        # Safe evaluation
        allowed_chars = set('0123456789+-*/().^ ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression.replace('^', '**'))
        return str(result)
    except Exception as e:
        return f"Error: {e}"

def get_weather(city: str) -> str:
    """Get weather for a city (mock)."""
    # Mock weather data
    weather_data = {
        "new york": {"temp": 72, "condition": "sunny"},
        "london": {"temp": 58, "condition": "cloudy"},
        "tokyo": {"temp": 68, "condition": "partly cloudy"},
        "paris": {"temp": 65, "condition": "rainy"},
    }
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        return f"Weather in {city}: {data['temp']}°F, {data['condition']}"
    return f"Weather data not available for {city}"

def search_web(query: str) -> str:
    """Search the web (mock)."""
    # Mock search results
    return f"Search results for '{query}': Found 3 relevant articles about {query}."

# Create tool definitions
tools = [
    ToolDefinition(
        name="calculator",
        description="Evaluate mathematical expressions. Use for any calculations.",
        parameters={
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Math expression to evaluate (e.g., '2+2', '15*23')"
                }
            },
            "required": ["expression"]
        },
        function=calculator
    ),
    ToolDefinition(
        name="get_weather",
        description="Get current weather for a city.",
        parameters={
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "City name (e.g., 'New York', 'London')"
                }
            },
            "required": ["city"]
        },
        function=get_weather
    ),
    ToolDefinition(
        name="search_web",
        description="Search the web for information.",
        parameters={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search query"
                }
            },
            "required": ["query"]
        },
        function=search_web
    ),
]

print(f"Defined {len(tools)} tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

## 3. Building a Simple Agent

Let's build an agent that can use our tools.

In [ ]:
class SimpleAgent:
    """A simple tool-using agent."""
    
    def __init__(
        self,
        llm: LLMClient,
        tools: list[ToolDefinition],
        system_prompt: str = None,
        max_iterations: int = 5,
    ):
        self.llm = llm
        self.tools = {t.name: t for t in tools}
        self.tool_schemas = [t.to_schema() for t in tools]
        self.max_iterations = max_iterations
        
        self.system_prompt = system_prompt or """You are a helpful assistant with access to tools.
Use tools when needed to answer questions accurately.
Always explain your reasoning before using a tool."""
    
    async def run(self, user_input: str) -> str:
        """Run the agent on user input."""
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_input},
        ]
        
        for iteration in range(self.max_iterations):
            print(f"\n--- Iteration {iteration + 1} ---")
            
            # Get LLM response with tool calling
            response = await self.llm.chat(
                messages=messages,
                tools=self.tool_schemas,
            )
            
            # Check if the model wants to call a tool
            if response.tool_calls:
                for tool_call in response.tool_calls:
                    tool_name = tool_call.function.name
                    tool_args = json.loads(tool_call.function.arguments)
                    
                    print(f"🔧 Calling tool: {tool_name}")
                    print(f"   Arguments: {tool_args}")
                    
                    # Execute the tool
                    if tool_name in self.tools:
                        result = self.tools[tool_name].function(**tool_args)
                        print(f"   Result: {result}")
                    else:
                        result = f"Error: Unknown tool {tool_name}"
                    
                    # Add tool result to messages
                    messages.append({
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [tool_call.model_dump()]
                    })
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": result,
                    })
            else:
                # No tool calls - return final response
                print(f"✅ Final response")
                return response.content
        
        return "Max iterations reached without final answer."

# Create agent
agent = SimpleAgent(llm=llm, tools=tools)
print("Agent created!")

In [ ]:
# Test the agent
result = await agent.run("What is 15 * 23?")
print(f"\n📝 Answer: {result}")

In [ ]:
# Test with weather
result = await agent.run("What's the weather like in Tokyo?")
print(f"\n📝 Answer: {result}")

In [ ]:
# Test multi-tool query
result = await agent.run(
    "Compare the weather in New York and London, then calculate the temperature difference."
)
print(f"\n📝 Answer: {result}")

## 4. ReAct Agent Pattern

ReAct (Reasoning + Acting) is a powerful agent architecture.

In [ ]:
REACT_SYSTEM_PROMPT = """You are an AI assistant that follows the ReAct pattern.

For each step, you will:
1. THOUGHT: Reason about what you need to do next
2. ACTION: Call a tool if needed
3. OBSERVATION: Review the tool result
4. Repeat until you have the final answer

Available tools:
{tools}

When you have the final answer, respond with:
FINAL ANSWER: [your answer]

Always explain your reasoning clearly."""

class ReActAgent:
    """ReAct-style agent with explicit reasoning."""
    
    def __init__(
        self,
        llm: LLMClient,
        tools: list[ToolDefinition],
        max_iterations: int = 10,
    ):
        self.llm = llm
        self.tools = {t.name: t for t in tools}
        self.max_iterations = max_iterations
        
        # Build tool descriptions
        tool_desc = "\n".join([
            f"- {t.name}: {t.description}" for t in tools
        ])
        self.system_prompt = REACT_SYSTEM_PROMPT.format(tools=tool_desc)
    
    async def run(self, user_input: str, verbose: bool = True) -> str:
        """Run the ReAct agent."""
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"Task: {user_input}"},
        ]
        
        for iteration in range(self.max_iterations):
            if verbose:
                print(f"\n{'='*50}")
                print(f"Step {iteration + 1}")
                print("="*50)
            
            # Get next action
            response = await self.llm.chat(
                messages=messages,
                tools=[t.to_schema() for t in self.tools.values()],
            )
            
            content = response.content or ""
            if verbose and content:
                print(f"\n🤔 {content}")
            
            # Check for final answer
            if "FINAL ANSWER:" in content:
                answer = content.split("FINAL ANSWER:")[1].strip()
                return answer
            
            # Execute tool calls
            if response.tool_calls:
                for tool_call in response.tool_calls:
                    tool_name = tool_call.function.name
                    tool_args = json.loads(tool_call.function.arguments)
                    
                    if verbose:
                        print(f"\n🔧 ACTION: {tool_name}({tool_args})")
                    
                    # Execute
                    if tool_name in self.tools:
                        result = self.tools[tool_name].function(**tool_args)
                    else:
                        result = f"Unknown tool: {tool_name}"
                    
                    if verbose:
                        print(f"📊 OBSERVATION: {result}")
                    
                    # Update messages
                    messages.append({
                        "role": "assistant",
                        "content": content,
                        "tool_calls": [tool_call.model_dump()]
                    })
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": f"OBSERVATION: {result}",
                    })
            else:
                # No tool calls, continue reasoning
                messages.append({"role": "assistant", "content": content})
                messages.append({"role": "user", "content": "Continue with your next thought or provide FINAL ANSWER:"})
        
        return "Could not complete task within iteration limit."

# Create ReAct agent
react_agent = ReActAgent(llm=llm, tools=tools)
print("ReAct agent created!")

In [ ]:
# Test ReAct agent
result = await react_agent.run(
    "I'm planning a trip. Check the weather in Paris and Tokyo, "
    "calculate how much warmer one is than the other, and recommend which to visit."
)

print(f"\n{'='*50}")
print("FINAL RESULT:")
print("="*50)
print(result)

## 5. Using AI Beast's Agent Module

AI Beast provides a full-featured agent implementation.

In [ ]:
from modules.agent import Agent, AgentConfig, ToolRegistry

# Register tools
registry = ToolRegistry()

@registry.register(
    name="calculator",
    description="Evaluate mathematical expressions",
)
def calc(expression: str) -> str:
    """Calculate math expressions."""
    return calculator(expression)

@registry.register(
    name="weather",
    description="Get weather for a city",
)
def weather(city: str) -> str:
    """Get weather information."""
    return get_weather(city)

# Create agent with config
beast_agent = Agent(
    config=AgentConfig(
        name="assistant",
        model="llama3.2",
        system_prompt="You are a helpful AI assistant.",
        max_iterations=10,
        verbose=True,
    ),
    tool_registry=registry,
)

print("AI Beast agent ready!")

In [ ]:
# Run the agent
response = await beast_agent.run(
    "What's 25 * 17 and what's the weather in New York?"
)

print(f"\n\nFinal Answer: {response.content}")

## Next Steps

Congratulations! You've built AI agents with tools.

Continue with:
- **[05_advanced_agents.ipynb](05_advanced_agents.ipynb)** - Multi-agent systems
- **[06_deployment.ipynb](06_deployment.ipynb)** - Deploy your agents

## Exercises

1. Create a tool that reads files from disk
2. Build an agent that can browse websites
3. Implement memory persistence across sessions
4. Create a multi-agent system where agents collaborate

## Tips

- **Clear tool descriptions**: Help the model understand when to use each tool
- **Handle errors gracefully**: Tools can fail, make sure to catch exceptions
- **Limit iterations**: Prevent infinite loops with max_iterations
- **Log everything**: Tracing helps debug agent behavior